## Identify Board Space

https://scikit-image.org/docs/stable/auto_examples/features_detection/plot_corner.html#sphx-glr-auto-examples-features-detection-plot-corner-py

### Detect Edges

In [ ]:
import itertools
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
from scipy import ndimage as ndi
from skimage import io, feature, morphology
from skimage.color import rgb2gray
from skimage.feature import canny 
from skimage import transform as tf
from skimage.filters import threshold_otsu
from skimage.transform import resize, hough_line, hough_line_peaks, warp, ProjectiveTransform
from skimage import img_as_ubyte
# import skimage.transform
# x = img_as_ubyte(skimage.transform.resize(y, (100, 100)))

def load_image(image_path, resize_img=True, grayscale_img=False):
    image = io.imread(image_path)

    if resize_img:
        # resize the shortest edge to 480
        if image.shape[1] < image.shape[0]:
            image = resize(
                image, 
                (image.shape[0] / (image.shape[1] / 480), 480),
            )
        else:
            image = resize(
                image, 
                (480, image.shape[1] / (image.shape[0] / 480)),
            )

    if grayscale_img:
        image = rgb2gray(image)

    return image


def intersection(L1, L2):
    D  = L1[0] * L2[1] - L1[1] * L2[0]
    Dx = L1[2] * L2[1] - L1[1] * L2[2]
    Dy = L1[0] * L2[2] - L1[2] * L2[0]
    if D != 0:
        x = Dx / D
        y = Dy / D
        return x,y
    else:
        return False


def detect_board(board_image, plot=True):
    image = rgb2gray(board_image)
    
    binary = image > threshold_otsu(image)  # to black & white
    edges = canny(binary)  # get edges
    filled = ndi.binary_fill_holes(edges)  # fill shapes

    # only keep shapes larger than 1/4 of the image area
    cleaned = morphology.remove_small_objects(
        filled, image.shape[0] * image.shape[1] / 4 
    )
    
    edge = canny(cleaned)  # get edges of large shape
    
    # get straight lines
    h, theta, d = hough_line(edge)
    _, angles, dists = hough_line_peaks(
        hspace=h, angles=theta, dists=d, num_peaks=4, 
        threshold=0.5
    )

    lines = []
    for angle, C in zip(angles, dists):
        # Ax + By = C
        A = np.cos(angle)
        B = np.sin(angle)
        lines.append((A, B, C))

    corners = []
    for L1, L2 in itertools.combinations(lines, 2):
        pt = intersection(L1, L2)
        conditions = [
             pt[0] > -50, 
             pt[1] > -50, 
             pt[0] < image.shape[1] + 50,
             pt[1] < image.shape[0] + 50, 
        ]
        if all(conditions):
            corners.append(pt)
    
    if plot:
        fig, ax = plt.subplots(nrows=3, ncols=2, figsize=(10, 12))
        ax[0][0].imshow(binary, cmap=plt.cm.gray)
        ax[0][1].imshow(edges, cmap=plt.cm.gray)
        ax[1][0].imshow(filled, cmap=plt.cm.gray)
        ax[1][1].imshow(cleaned, cmap=plt.cm.gray)
        ax[2][0].imshow(edge, cmap=plt.cm.gray)

        ax[2][1].imshow(board_image, cmap=plt.cm.gray)
        ax[2][1].plot([t[0] for t in corners], 
                      [t[1] for t in corners], '.r')

        for angle, dist in zip(angles, dists):
            y0 = (dist - 0 * np.cos(angle)) / np.sin(angle)
            y1 = (dist - image.shape[1] * np.cos(angle)) / np.sin(angle)
            ax[2][1].plot((0, image.shape[1]), (y0, y1), '-r')

        ax[2][1].set_xlim((0, image.shape[1]))
        ax[2][1].set_ylim((image.shape[0], 0))
        
        plt.tight_layout()
        plt.show()
    
    return corners

def average(ls):
    return sum(ls) / len(ls)

def sort_points(corners):
    centroid = (average([t[0] for t in corners]),
                average([t[1] for t in corners]))
    
    board_srt = [
        next(filter(lambda x: is_top_left(x, centroid), corners)),
        next(filter(lambda x: is_btm_left(x, centroid), corners)),
        next(filter(lambda x: is_btm_rght(x, centroid), corners)),
        next(filter(lambda x: is_top_rght(x, centroid), corners)),
    ]

    return board_srt

def is_top_left(pt, centroid):
    return pt[0] < centroid[0] and pt[1] < centroid[1]

def is_btm_left(pt, centroid):
    return pt[0] < centroid[0] and pt[1] > centroid[1]

def is_btm_rght(pt, centroid):
    return pt[0] > centroid[0] and pt[1] > centroid[1]

def is_top_rght(pt, centroid):
    return pt[0] > centroid[0] and pt[1] < centroid[1]

In [ ]:
_id = 2
img_paths = [x for x in Path('img/config_3').iterdir()]
image = load_image(str(img_paths[_id]))

In [ ]:
board = detect_board(image, plot=False)

In [ ]:
n_px = 480
src = np.array([[   0,    0], [   0, n_px], 
                [n_px, n_px], [n_px,    0]])
dst = np.array(sort_points(board))
tform3 = ProjectiveTransform()
tform3.estimate(src, dst)
warped = warp(image, tform3, output_shape=(n_px, n_px))
warped = img_as_ubyte(warped)

crop = 40
warped = warped[crop:-crop, crop:-crop]
warped = resize(warped, (99, 99))
plt.imshow(warped);

In [ ]:
# TODO:
# * bin photos into the same puzzle configuration, same perspective, different lighting
# * create pipeline to label photos, only need to label once per puzzle config, then rotate to match perspective

In [ ]:
def build_database():
    

In [ ]:
def ball_loc(image, x, y, ball_size):   
    ball = image[
        ball_size * y : ball_size * y + ball_size, 
        ball_size * x : ball_size * x + ball_size, 
    ]
    
    return ball

In [ ]:
ball_sz = warped.shape[0] // 9

for x in range(9):
    for y in range(9):
        ball = ball_loc(image, x, y, ball_sz)
        save_path = f'ball_{x}_{y}.png'
#         io.imsave(save_path, ball)

In [ ]:
plt.imshow(ball_loc(warped, 1, 7, warped.shape[0] // 9));

In [ ]:
plt.imshow(io.imread(save_path));

In [ ]:
io.imread(save_path)

In [ ]:
ball

In [ ]:
# TODOs:
# * compress image further
# * slice into small images centered around the holes
# * label images

## Exploratory Code

In [ ]:
# Compute the Canny filter for several values of sigma
edges = []
for sigma in [x/10 for x in range(11)]:
    edges.append(feature.canny(image, sigma=sigma))

In [ ]:
# display results
fig, axes = plt.subplots(nrows=len(edges)+1, ncols=1, figsize=(10, 10),
                         sharex=True, sharey=True)

axes[0].imshow(image, cmap=plt.cm.gray)
axes[0].set_title('original image', fontsize=15)

i = 0
for ax, edge in zip(axes[1:], edges):
    i += 1
    ax.imshow(edge, cmap=plt.cm.gray)
    ax.set_title(f'Canny filter, $\sigma={i}$', fontsize=15)

# fig.tight_layout()

plt.show();

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from skimage.filters import roberts, sobel, scharr, prewitt


edge_roberts = roberts(image)
edge_sobel = sobel(image)

fig, ax = plt.subplots(ncols=2, sharex=True, sharey=True,
                       figsize=(8, 4))

ax[0].imshow(edge_roberts, cmap=plt.cm.gray)
ax[0].set_title('Roberts Edge Detection')

ax[1].imshow(edge_sobel, cmap=plt.cm.gray)
ax[1].set_title('Sobel Edge Detection')

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
from skimage.filters import threshold_otsu


image = get_image(grayscale_img=True)
thresh = threshold_otsu(image)
binary = image > thresh

fig, axes = plt.subplots(ncols=2, figsize=(8, 2.5))

axes[0].imshow(image, cmap=plt.cm.gray)
axes[0].set_title('Original')

axes[1].imshow(binary, cmap=plt.cm.gray)
axes[1].set_title('Thresholded')

plt.show()

https://stackoverflow.com/questions/11602259/find-the-smallest-containing-convex-polygon-with-a-given-number-of-points
https://www.researchgate.net/publication/226436325_Minimum_area_circumscribing_Polygons

In [ ]:
detect_board()

In [ ]:
import numpy as np

from skimage.transform import hough_line, hough_line_peaks
from skimage.feature import canny
from skimage import data

import matplotlib.pyplot as plt
from matplotlib import cm



# Generating figure 1
fig, axes = plt.subplots(1, 2, figsize=(15, 6))
ax = axes.ravel()

ax[0].imshow(image, cmap=cm.gray)
ax[0].set_title('Input image')
ax[0].set_axis_off()

ax[1].imshow(image, cmap=cm.gray)
for _, angle, dist in zip(*hough_line_peaks(h, theta, d)):
    y0 = (dist - 0 * np.cos(angle)) / np.sin(angle)
    y1 = (dist - image.shape[1] * np.cos(angle)) / np.sin(angle)
    ax[1].plot((0, image.shape[1]), (y0, y1), '-r')
ax[1].set_xlim((0, image.shape[1]))
ax[1].set_ylim((image.shape[0], 0))
ax[1].set_axis_off()
ax[1].set_title('Detected lines')

plt.tight_layout()
plt.show();

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))

ax.imshow(image, cmap=cm.gray)
ax.plot([t[0] for t in corners], 
        [t[1] for t in corners],
        '.r')
plt.show();

In [ ]:
corners

In [ ]:
import math

def clockwiseangle_and_distance(point, refvec= [0, 1]):
    # Vector between point and the origin: v = p - o
    vector = [point[0]-origin[0], point[1]-origin[1]]
    # Length of vector: ||v||
    lenvector = math.hypot(vector[0], vector[1])
    # If length is zero there is no angle
    if lenvector == 0:
        return -math.pi, 0
    # Normalize vector: v/||v||
    normalized = [vector[0]/lenvector, vector[1]/lenvector]
    dotprod  = normalized[0]*refvec[0] + normalized[1]*refvec[1]     # x1*x2 + y1*y2
    diffprod = refvec[1]*normalized[0] - refvec[0]*normalized[1]     # x1*y2 - y1*x2
    angle = math.atan2(diffprod, dotprod)
    # Negative angles represent counter-clockwise angles so we need to subtract them 
    # from 2*pi (360 degrees)
    if angle < 0:
        return 2*math.pi+angle, lenvector
    # I return first the angle because that's the primary sorting criterium
    # but if two vectors have the same angle then the shorter distance should come first.
    return angle, lenvector

In [ ]:
origin = list(np.array(corners).mean(axis=0))
sorted(corners, key=clockwiseangle_and_distance)

In [ ]:
from skimage import transform as tf

# top left, bottom left, bottom right, top right
src = np.array([[0, 0], [0, 500], [500, 500], [500, 0]])
# TODO: automate the identification of these corners:
dst = np.array(corners)

tform3 = tf.ProjectiveTransform()
tform3.estimate(src, dst)
warped = tf.warp(image, tform3, output_shape=(500, 500))

fig, ax = plt.subplots(nrows=2, figsize=(10, 10))

ax[0].imshow(image, cmap=plt.cm.gray)
ax[0].plot(dst[:, 0], dst[:, 1], '.r')
ax[1].imshow(warped, cmap=plt.cm.gray)

plt.tight_layout()

plt.show()

## Using Contours

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from skimage import measure
from skimage.filters import threshold_otsu


def bbox_area(points):
    """X: 2D array"""
    if len(points.shape) != 2 or points.shape[1] != 2:
        raise ValueError(
            f"Points must be a (n,2), array but it has shape {points.shape}"
        )
    if points.shape[0] < 1:
        raise ValueError("Can't compute bounding box for empty coordinates")
    minx, miny = np.min(points, axis=0)
    maxx, maxy = np.max(points, axis=0)

    return (maxx - minx) * (maxy - miny)

image = get_image(grayscale_img=True)

thresh = threshold_otsu(image)
binary = image > thresh

# Find contours at a constant value of 0.1
contours = measure.find_contours(binary, 0.1)
# Get largest contour
contour = max(contours, key=bbox_area)

# Display the image and plot largest contour bounding box
fig, ax = plt.subplots(figsize=(10,10))
ax.imshow(img, interpolation='nearest', cmap=plt.cm.gray)
ax.plot(contour[:, 1], contour[:, 0], linewidth=4)

plt.show()

In [ ]:
approximate_polygon?

In [ ]:
from skimage.measure import approximate_polygon, subdivide_polygon

appr_contour = approximate_polygon(contour, tolerance=20)

print(contour.shape, appr_contour.shape)

# Display the image and plot largest contour bounding box
fig, ax = plt.subplots(figsize=(10,10))
ax.imshow(img, interpolation='nearest', cmap=plt.cm.gray)
ax.plot(appr_contour[:, 1], appr_contour[:, 0], linewidth=4)

plt.show()

### Identify Corners

In [ ]:
from pathlib import Path

from matplotlib import pyplot as plt

from skimage.feature import corner_harris, corner_subpix, corner_peaks

image = get_image()

In [ ]:
coords = corner_peaks(corner_harris(image), min_distance=5)
coords_subpix = corner_subpix(image, coords, window_size=13)

fig, ax = plt.subplots()
ax.imshow(image, interpolation='nearest', cmap=plt.cm.gray)
# ax.plot(coords[:, 1], coords[:, 0], '.b', markersize=3)
ax.plot(coords_subpix[:, 1], coords_subpix[:, 0], '+r', markersize=15)
plt.show()

## Rectify Board Space

https://scikit-image.org/docs/stable/auto_examples/applications/plot_geometric.html#sphx-glr-auto-examples-applications-plot-geometric-py

TODO: automate identification of board corners

In [ ]:
from pathlib import Path

import math
import numpy as np
import matplotlib.pyplot as plt

from skimage import io
from skimage import data
from skimage import transform as tf

img_paths = [x for x in Path('img').iterdir()]
board = io.imread(str(img_paths[0]))

fig, ax = plt.subplots(figsize=(8, 8))
ax.imshow(board);

In [ ]:
# top left, bottom left, bottom right, top right
src = np.array([[0, 0], [0, 500], [500, 500], [500, 0]])
# TODO: automate the identification of these corners:
dst = np.array([
    [1250,   420], [930,  2420],  # x, y left edge
    [3590,  2490], [3260,  410],   # x, y right edge
])

tform3 = tf.ProjectiveTransform()
tform3.estimate(src, dst)
warped = tf.warp(board, tform3, output_shape=(500, 500))

fig, ax = plt.subplots(nrows=2, figsize=(10, 10))

ax[0].imshow(board, cmap=plt.cm.gray)
ax[0].plot(dst[:, 0], dst[:, 1], '.r')
ax[1].imshow(warped, cmap=plt.cm.gray)

plt.tight_layout()

plt.show()

### Resources

* https://github.com/EdjeElectronics/TensorFlow-Object-Detection-API-Tutorial-Train-Multiple-Objects-Windows-10
* https://github.com/EdjeElectronics/TensorFlow-Object-Detection-on-the-Raspberry-Pi
* https://hackernoon.com/building-an-insanely-fast-image-classifier-on-android-with-mobilenets-in-tensorflow-dc3e0c4410d4
* http://matpalm.com/blog/counting_bees/
* https://www.amazon.com/gp/product/B01ER2SKFS
* https://github.com/tensorflow/models/tree/master/research/slim/nets/mobilenet
* https://www.reddit.com/r/MachineLearning/comments/8dy6wi/p_live_object_detection_on_raspberry_pi_cpu_with/